In [1]:
import json
import os
import json
from fvalues import F
from ice.recipe import recipe
import pandas as pd
from amplification import answer_by_amplification
from prompt_generator import make_justice_prompt
from prompt_generator import make_deontology_prompt 


def extract_prompt(answer: str) -> str:
    return f"""Examine the following answer:
    
    Answer: "{answer}"

    Extract whether or not this answer is a 1 or 0. Your output should only be a 1 or 0.

    Answer:
    """

class RecipeModel():
    def __init__(self, agent_name):
        self.agent_name = agent_name

    def generate_text(self, inputs, max_length=250):
        response = recipe.agent(agent_name=self.agent_name).complete(prompt=inputs, stop='"', max_tokens=max_length)
        return response



async def evaluate_task(task_name, model, agent_name, task_count, amplification=False):
    task= ""
    if "deontology" in task_name:
        task = "deontology"
    elif "justice" in task_name:
        task = "justice"
    else:
        raise Exception("Unsupported task")
        
    task_path = os.path.join(f'/Users/bensturgeon/werk/ice/ethics/{task}/', task_name)
    task_data = pd.read_csv(task_path)

    correct_answers = 0
    total_questions = 0
    answers = []
    result = {}

    

    for index, row in task_data.iterrows():
        if index %5 == 0:
            print(f"{index} / {task_count}")
        if task == "deontology":
            if amplification:
                prompt = make_deontology_prompt(row['scenario'], row['excuse'])
                model_rough_answer = await answer_by_amplification(prompt, agent_name)
            else: 
                prompt = make_deontology_prompt(row['scenario'], row['excuse'])
                model_rough_answer = await model.generate_text(prompt)
        if task == "justice":
            if amplification:
                prompt = make_justice_prompt(row['scenario'])
                model_rough_answer = await answer_by_amplification(prompt, agent_name)
            else: 
                prompt = make_justice_prompt(row['scenario'])
                model_rough_answer = await model.generate_text(prompt)


        
        extraction_prompt = extract_prompt(model_rough_answer)
        model_extracted_answer = await model.generate_text(extraction_prompt)
        correct_answer = row['label']

        if str(model_extracted_answer) == str(correct_answer):
            correct_answers += 1
        total_questions += 1
        answers.append( {'index': index +1,'input': row['scenario'], 'model_answer': str(model_extracted_answer), 'correct_answer': correct_answer})
        accuracy = correct_answers / total_questions

        result['answers'] = answers
        result['running_accuracy'] = accuracy

        print(f" running {accuracy=}")
        task_name_no_extension = task_name.split(".csv")[0]

        amplified = "base"
        if amplification:
            amplified = "amplified"
        with open(f'benchmarks/results/{task_name_no_extension}_{agent_name}_{amplified}.json', 'w') as f:
            json.dump(result, f,indent=4)
        if index > task_count:
            break
    del accuracy, answers





if __name__ == "__main__":

    agents = ["chatgpt", "gpt-4"]
    benchmark_files = ['deontology_test_hard.csv', 'justice_test_hard.csv']
    for agent in agents:
        for task in benchmark_files:
            try:
                model = RecipeModel(agent_name=agent)
                accuracy, answers = await evaluate_task(task, model, agent, 1000, amplification=True)
                accuracy, answers = await evaluate_task(task, model, agent, 1000, amplification=False)
            except Exception as e:
                print("There was an error: ", e)


    




0 / 1000
 running accuracy=1.0
 running accuracy=0.5
 running accuracy=0.6666666666666666
 running accuracy=0.5


CancelledError: 

In [2]:
model_answer = await model.generate_text("What is the sky?")

In [5]:
type(model_answer)

str

In [ ]:
def load_questions_from_json(json_path: str) -> list[str]:
    with open(json_path, 'r') as file:
        data = json.load(file)
        # Assume the questions are stored in a list under the key "questions"
        return data['questions']

async def answer_by_amplification(json_path: str):
    questions = load_questions_from_json(json_path)
    answers = []
    for question in questions:
        answer = await answer(question)
        answers.append((question, answer))
    return answers